# Random Forest Classifier with Pipeline and Hyperparameter Tuning 

In [25]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [26]:
df.isnull().sum() # Checking the missing values

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [27]:
df.describe() #EDA 

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [28]:
# Converting the time from categorical to numerical feature via lable encoder
df.time 

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [29]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [30]:
df.time.unique()

array([0, 1])

In [31]:
# Converting the entire data into dependent and independent features
X=df.drop(labels=['time'],axis=1)
y=df.time

In [32]:
X.head

<bound method NDFrame.head of      total_bill   tip     sex smoker   day  size
0         16.99  1.01  Female     No   Sun     2
1         10.34  1.66    Male     No   Sun     3
2         21.01  3.50    Male     No   Sun     3
3         23.68  3.31    Male     No   Sun     2
4         24.59  3.61  Female     No   Sun     4
..          ...   ...     ...    ...   ...   ...
239       29.03  5.92    Male     No   Sat     3
240       27.18  2.00  Female    Yes   Sat     2
241       22.67  2.00    Male    Yes   Sat     2
242       17.82  1.75    Male     No   Sat     2
243       18.78  3.00  Female     No  Thur     2

[244 rows x 6 columns]>

In [33]:
y.head

<bound method NDFrame.head of 0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64>

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [35]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [36]:
# Pipelining the Entire Process to make it easy 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # Handle Missing Values 
from sklearn.preprocessing import StandardScaler # For Feature Scaling 
from sklearn.preprocessing import OneHotEncoder #Categorical to Number
from sklearn.compose import ColumnTransformer

In [37]:
categorical_col=['sex','smoker','day']
numerical_col=['total_bill','tip','size']

In [38]:
# To automate the Feature Engineering 

# Numerical Pipeline for Feature Engineering 
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), # for handling missing values replacing by median
        ('scaler',StandardScaler()) # Applying Standard Scler
    ]
)

#Categorical Pipeline 
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), # Handling Missing Values
                ('onehotencoder',OneHotEncoder())
                ]
             )

In [39]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_col),
    ('cat_pipeline',cat_pipeline,categorical_col)
])

In [40]:
# Applying the preprocessor in the entire dataset
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [51]:
## Model Training Automation
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()

}

In [55]:
from sklearn.metrics import accuracy_score

In [56]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)

            

        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction
       
        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score
            

            
    return report


In [57]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [59]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [60]:
classfier=RandomForestClassifier()

In [61]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [62]:
from sklearn.model_selection import RandomizedSearchCV

In [63]:
cv=RandomizedSearchCV(classfier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.974 total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.923 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.923 total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.949 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END criter

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [64]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 10, 'criterion': 'entropy'}

In [65]:
## RandomForest Regression Solve it Internal Assignment